# ANALYZE COMPANY'S SALES

In [1]:
# Load all the needed packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import datetime
import scipy.stats as sci
%matplotlib inline

In [2]:
transactions = pd.read_csv("datasets/transactions.csv")
customers = pd.read_csv("datasets/customers.csv")
products = pd.read_csv("datasets/products.csv")

In [3]:
transactions.head(2)

,id_prod,date,session_id,client_id
0,0_1483,2021-04-10 18:37:28.723910,s_18746,c_4450
1,2_226,2022-02-03 01:55:53.276402,s_159142,c_277


In [4]:
transactions.info(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337016 entries, 0 to 337015
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id_prod     337016 non-null  object
 1   date        337016 non-null  object
 2   session_id  337016 non-null  object
 3   client_id   337016 non-null  object
dtypes: object(4)
memory usage: 10.3+ MB


In [5]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8623 entries, 0 to 8622
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   client_id  8623 non-null   object
 1   sex        8623 non-null   object
 2   birth      8623 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 202.2+ KB


In [7]:
customers.head(2)

,client_id,sex,birth
0,c_4410,f,1967
1,c_7839,f,1975


In [9]:
products.head(2)

,id_prod,price,categ
0,0_1421,19.99,0
1,0_1368,5.13,0


In [10]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3287 entries, 0 to 3286
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id_prod  3287 non-null   object 
 1   price    3287 non-null   float64
 2   categ    3287 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 77.2+ KB


In [11]:
products.describe()

,price,categ
count,3287.000000,3287.000000
mean,21.856641,0.370246
std,29.847908,0.615387
min,-1.000000,0.000000
25%,6.990000,0.000000
50%,13.060000,0.000000
75%,22.990000,1.000000
max,300.000000,2.000000


## Cleaning all datasets

### Cleaning transactions dataset  
#### Define

- **Some date starting with test must be split**
- **Date variable must be a *datetime* not a string**  
- **We must split id_prod in category and id_prod**
- **We must split session_id in session id and session_category**
- **We must split client_id in client_id and client_category**
- **Turn all the categories variable into a category data type**

#### Code

In [12]:
# Write  a function to split a column
def split_columns(dataset, col):
    ### This function take a dataset and a column of the dataset split the column and return the 2 new columns
    new_col_1 = dataset[col].apply(lambda x : x.split("_")[1])
    new_col_2 = dataset[col].apply(lambda x : x.split("_")[0].upper())
    return new_col_1, new_col_2

In [13]:
# Create a copy of transactions dataset
transactions_df = transactions.copy()

In [14]:
# Split id_prod in 2 columns,id_prod and category
transactions_df["id_prod"], transactions_df["category"] = split_columns(transactions_df, "id_prod")

In [15]:
# Split client_id columns into 2 columns, client_id and client_category
transactions_df["client_id"], transactions_df["client_category"] = split_columns(transactions_df, "client_id")

In [16]:
# Split session_id in 2 columns, session_id and sesseion_category
transactions_df["session_id"], transactions_df["session_category"] = split_columns(transactions_df, "session_id")

In [18]:
# Check if everything is ok
transactions_clean.head(2)

,id_prod,date,session_id,client_id,category,client_category,session_category
0,1483,2021-04-10 18:37:28.723910,18746,4450,0,C,S
1,226,2022-02-03 01:55:53.276402,159142,277,2,C,S


In [19]:
# Check the different categories
transactions_clean.query("category == 'T'")

,id_prod,date,session_id,client_id,category,client_category,session_category
1431,0,test_2021-03-01 02:30:02.237420,0,1,T,CT,S
2365,0,test_2021-03-01 02:30:02.237446,0,1,T,CT,S
2895,0,test_2021-03-01 02:30:02.237414,0,1,T,CT,S
5955,0,test_2021-03-01 02:30:02.237441,0,0,T,CT,S
7283,0,test_2021-03-01 02:30:02.237434,0,1,T,CT,S
...,...,...,...,...,...,...,...
332594,0,test_2021-03-01 02:30:02.237445,0,0,T,CT,S
332705,0,test_2021-03-01 02:30:02.237423,0,1,T,CT,S
332730,0,test_2021-03-01 02:30:02.237421,0,1,T,CT,S
333442,0,test_2021-03-01 02:30:02.237431,0,1,T,CT,S


There are 200 rows which date starts with test. We can guess that it was just to *test* if the system is working or not. These rows are not useful for our analysis. We will remove them.
We can therefore notice that the test day was on 2021-03-01 at 02:30:02 am.

In [20]:
# Remove all the test dates
transactions_clean = transactions_clean.query("category != 'T'")

In [21]:
# Check if there are still test date, no output means there is no test date anymore
assert transactions_clean.category.all() != "T"

In [ ]:
transactions_clean.date = transactions_clean.date.astype("datetime64")

In [ ]:
# Assert that the date is in the correct type
transactions_clean.date.head()

In [ ]:
# turn all the categories variable into a category data type
transactions_clean.iloc[:, 4:] = transactions_clean.iloc[:, 4:].astype("category")

In [ ]:
# Check if the variables are categories
transactions_clean.info()

### Cleaning customers dataset  
#### Define  
- **Split client_id variable into 2 variables**
- **Turn sex variabble in uppercase**
- **Turn sex variable into category data type**

In [ ]:
# Make a copy of customers dataset
customers_clean = customers.copy()

In [ ]:
# Spllit client_id
customers_clean["client_id"], customers_clean["client_category"] = split_columns(customers_clean, "client_id")

In [ ]:
# Turn sex in uppercase
customers_clean.sex = customers_clean.sex.map(lambda x : x.upper())

In [ ]:
# Turn sex into category
customers_clean.sex = customers_clean.sex.astype("category")

In [ ]:
# Check the final dataset
print(customers_clean.head())
print("=========================================")
customers_clean.info()

### Cleaning products dataset  
#### Define  
- **Categ variable must be category type not an int**
- **Change the categ name to category**  
- **Split the id_prod and keep just the id products**  
- **There is a price of -1, we will remove it**

In [ ]:
# Make a copy of the products dataset
products_clean = products.copy()

In [ ]:
# Change the categ name to category
products_clean = products_clean.rename(columns={"categ":"category"})

In [ ]:
# Split id_prod and keep just the id products using the split_columns function
products_clean["id_prod"], products_clean["category"] = split_columns(products_clean, "id_prod")

In [ ]:
# Turn the category into a category type
products_clean.category = products_clean.category.astype("category")

In [ ]:
# Check if everything is correct
products_clean.head(2)

In [ ]:
products_clean.info()

In [ ]:
# Check the -1 price row
products_clean.query("price == -1")

This is probably another test, we will remove it for a better analysis.

In [ ]:
products_clean = products_clean.query("price != -1")

In [ ]:
# Check the description to see if it is ok
products_clean.describe()

Now that all the datasets are clean we can join them all together in a unique dataset for anaylysis.

In [ ]:
# Join all the datasets together
sales_merge = transactions_clean.merge(products_clean, on = ["category", "id_prod"], how = "left")

In [ ]:
sales_df = sales_merge.merge(customers_clean, on = ["client_id", "client_category"], how = "left")

In [ ]:
# Turn category and client category into categorical variable
sales_df.iloc[:, 4:6] = sales_df.iloc[:, 4:6].astype("category")
# Check if it is ok
sales_df.info()

We can see that price has less values than the other columns, probably due to missing values. Let us confirm that.

In [ ]:
# Check the columns with missing values
sales_df.isna().any()

In [ ]:
# Plot the missing values
sales_df.isna().sum().plot(kind = "bar");

In [ ]:
# Check the id product with missing values
sales_df[sales_df.price.isna()]["id_prod"].unique()

There is just one product, probably the price was not availaible for that product.

In [ ]:
# Replace the NaN values with the median price
sales_df.price.fillna(sales_df.price.median(), inplace = True)

In [ ]:
# Assert that there are no missing values anymore in the dataframe
assert sales_df.isna().all().all() == False

In [ ]:
# Create the variable age, max date - birth date
actual_year = sales_df.date.max().year # Find the max year in the dataframe to use as actual year
sales_df["age"] = actual_year - sales_df.birth # Can lead to error if data is not updated over time
sales_df.head(2)

In [ ]:
# Remove the useless column birth
sales_clean = sales_df.drop("birth", axis = 1)
sales_clean.head(2)

## ANALYZE THE DATA  

Now that all the datasets are cleaned we can make some analysis. 


**What is the average price of the sales?**

In [ ]:
sales_clean.price.describe()

We have an average price of **17.22** but with a standard deviation of **17.85** which means that there are too much variability between the prices, there are probably outliers. Let's check for the IQR.  

In [ ]:
# IQR of the price
def iqr(column) -> float:
    """return the iqr of a given column"""
    return column.quantile(.75) - column.quantile(.25)
# IQR of the price
iqr(sales_clean.price)

We have an IQR of around **10**, the variability is high. Let's make a plot.

In [ ]:
# Boxplot to see the variability
sns.boxplot(sales_clean.price);

As we can see there are many outliers. Let's try to find them all. 

In [ ]:
# Find all the positive outliers
outliers = sales_clean.price.quantile(0.75)+1.5*iqr(sales_clean.price)
sales_clean.price[sales_clean.price > outliers].sort_values()

All the price between **34.77** to **300** are outliers. 

In [ ]:
# Boxplot of the outliers values
sales_clean_no_outliers = sales_clean[sales_clean.price < outliers]
plt.hist(sales_clean_no_outliers.price);

In [ ]:
sales_clean_no_outliers.price.agg([np.mean, np.median])

If we remove all the outliers the mean and median are almost the same and are around $13. 

In [ ]:
# check the proportion of outliers in the dataframe
sales_clean_outliers = sales_clean.price[sales_clean.price > outliers]
prop_outliers = len(sales_clean_outliers) / len(sales_clean) * 100
prop_outliers

The outliers represent about **6%** of the number of sales. Let see what it represent as value

In [ ]:
# total sales for the outliers
outliers_total_sales = sales_clean_outliers.sum() / sales_clean.price.sum() * 100
outliers_total_sales

Even if the outliers represent just **6%** of the sales, they represent **25%** of the total purchases amount.

In [ ]:
# mean, median and mode of the price in the dataframe
print("mean : ",sales_clean.price.mean())
print("median : ",sales_clean.price.median())
print("mode : ",sales_clean.price.mode()[0])

We confirm here that most of the sales are between **0 and 16**, with highest sales at **13.9**

**Which are the categories with more sales?**

In [ ]:
plt.figure(figsize = [8,6])
# Choose a standard color for the plot
base_color = sns.color_palette()[0]
# Barplot of the categories
sns.countplot(data = sales_clean, x = "category", color = base_color)
plt.title("Number of Sales per Products Category", fontsize = 15)
plt.xlabel(xlabel = "Category", fontsize = 12)
plt.ylabel(ylabel = "Number of Sales", fontsize = 12)
# Add some annotations
number_points = sales_clean.shape[0]        # number of points in the dataset
category_counts = sales_clean.category.value_counts() # all Values of the categories
locs, labels = plt.xticks()   # Current tick locations and labels
# Loop through each pair of locations and labels  
for loc, label in zip(locs, labels):
    # Text property for the label to get the current count
    count = category_counts[label.get_text()]
    percentage = '{:0.1f}%'.format(100*count/number_points) # Percentage of each category
    # Put the annotations inside the bar on the top
    plt.text(x = loc, y = count-10000, s = percentage, ha = 'center', color = 'orange')

**62%** of the sales are in category 0, **33%** in category 1 and just **5.2%** in category 2. 

**What is the Average sale in each category?**

In [ ]:
# Considering all the dataframe
plt.figure(figsize = [12,14])
plt.subplot(3,1,1)
sns.boxplot(data = sales_clean[sales_clean.category == "0"], x = "price")
plt.title("Category 0", fontsize = 14, fontweight = "bold")
plt.xlabel("")

plt.subplot(3,1,2)
sns.boxplot(data = sales_clean[sales_clean.category == "1"], x = "price")
plt.title("Category 1", fontsize = 14, fontweight = "bold")
plt.xlabel("")

plt.subplot(3,1,3)
sns.boxplot(data = sales_clean[sales_clean.category == "2"], x = "price")
plt.title("Category 2", fontsize = 14, fontweight = "bold")
plt.xlabel("Price", fontsize = 12, fontweight = "bold");

In [ ]:
for i in range(3):
    print("Mean Category " + str(i) + ": ", sales_clean.price[sales_clean.category == str(i)].mean())
    print("Median Category " + str(i) + ": ", sales_clean.price[sales_clean.category == str(i)].median())
    print("Mode Category " + str(i) + ": ", sales_clean.price[sales_clean.category == str(i)].mode()[0])
    print("Standard Deviation Category " + str(i) + ":", sales_clean.price[sales_merge.category == str(i)].std(),"\n")    

In [ ]:
# Average sales in each category removing outliers
plt.figure(figsize = [12,14])
plt.subplot(3,1,1)
sns.boxplot(data = sales_clean_no_outliers[sales_clean_no_outliers.category == "0"], x = "price")
plt.title("Category 0", fontsize = 14, fontweight = "bold")
plt.xlabel("")

plt.subplot(3,1,2)
sns.boxplot(data = sales_clean_no_outliers[sales_clean_no_outliers.category == "1"], x = "price")
plt.title("Category 1", fontsize = 14, fontweight = "bold")
plt.xlabel("")

plt.subplot(3,1,3)
sns.boxplot(data = sales_clean_no_outliers[sales_clean_no_outliers.category == "2"], x = "price")
plt.title("Category 2", fontsize = 14, fontweight = "bold")
plt.xlabel("Price", fontsize = 12, fontweight = "bold");

In [ ]:
# measures of center df without outliers
for i in range(3):
    print("Mean Category " + str(i) + ": ", sales_clean_no_outliers.price[sales_clean_no_outliers.category == str(i)].mean())
    print("Median Category " + str(i) + ": ", sales_clean_no_outliers.price[sales_clean_no_outliers.category == str(i)].median())
    print("Mode Category " + str(i) + ": ", sales_clean_no_outliers.price[sales_clean_no_outliers.category == str(i)].mode()[0])
    print("Standard Deviation Category " + str(i) + ":", sales_clean_no_outliers.price[sales_merge.category == str(i)].std(),"\n")

It seems that all the outiers values are from product of category 2. Product of category 2 are the most expensives.

**Distribution of sales by Age**

In [ ]:
bin_num = int(1 + np.log2(sales_clean.shape[0]))
plt.hist(sales_clean.age, bins = bin_num)
plt.xlabel("Age")
plt.title("Distribution of Sales by Age");

Most of the sales come from people aged between 30 and 55 with a peak at 42 years old.

### Sales per Gender

In [ ]:
plt.figure(figsize = [8,6])
# Choose a standard color for the plot
base_color = sns.color_palette()[0]
# Barplot of the categories
sns.countplot(data = sales_clean, x = "sex", color = base_color)
plt.title("Number of Sales per Gender", fontsize = 15)
plt.xlabel(xlabel = "Gender", fontsize = 12)
plt.ylabel(ylabel = "Number of Sales", fontsize = 12)
# Add some annotations
number_points = sales_clean.shape[0]        # number of points in the dataset
category_counts = sales_clean.sex.value_counts() # all Values of each category
locs, labels = plt.xticks()   # Current tick locations and labels
# Loop through each pair of locations and labels  
for loc, label in zip(locs, labels):
    # Text property for the label to get the current count
    count = category_counts[label.get_text()]
    percentage = '{:0.1f}%'.format(100*count/number_points) # Percentage of each category
    # Put the annotations inside the bar on the top
    plt.text(x = loc, y = count-10000, s = percentage, ha = 'center', color = 'orange')

The number of sales is almost the same for each gender.

**Sales over time**

Check all the date availaible.

In [ ]:
# Check the years 
sales_clean.date.dt.year.unique()

In [ ]:
# Check the last date of 2022
sales_clean.date.max()

We have data for 2021 and 2022. For 2022 we have just January and February.

In [ ]:
# Time series of the price each month just for 2021
sales_clean_21 = sales_clean[sales_clean.date.dt.year == 2021]
sales_clean_22 = sales_clean[sales_clean.date.dt.year == 2022]
g = sales_clean_21.groupby(sales_clean_21.date.dt.strftime("%b")).sum().reset_index() # We group by months of 2022
g1 = sales_clean_22.groupby(sales_clean_22.date.dt.strftime("%b")).sum().reset_index() # We group by months of 2022
fig, ax = plt.subplots()
ax.plot(g.date, g.price)
ax.plot(g1.date, g1.price);

In 2021 December is the month with the most sales, while October is the worse. Overall february is the month with the most sales.

**Concentration of prices**

In [ ]:
# Plot the lorenz curve
n = len(sales_clean.price) #size of the sample
lorenz = np.cumsum(np.sort(sales_clean.price) / sales_clean.price.sum()) #result will be an array
lorenz = np.append([0], lorenz) #values of the y axis on the curve, the curve will begin at 0

# plt.axes().axis("equal") #x and y axis will have the same length
xaxis = np.linspace(0-1/n, 1+1/n, n+1) #values of the x axis on the curve
plt.plot(xaxis, lorenz, drawstyle = "steps-post")
plt.plot(xaxis, xaxis, linestyle="--") #Plot the 1st bissector
plt.show()

In [ ]:
# Gini coefficient
AUC = (lorenz.sum() - lorenz[-1]/2 - lorenz[0]/2)/n # Area under the curve
S = 0.5 - AUC
gini = round(2*S,1)*100
gini

We have a gini coefficient of around 40%, which means that the level of inequality is not that high.

**Correlations**

In [ ]:
# Correlation between gender and categories of products purchased
sns.catplot(x = "sex", y = "price", col = "category", kind = "violin" , data = sales_clean);

We can see that in each category gender are almost simmetrical, this means that there is no difference of purchase between gender in each category.

In [ ]:
# Contigency table
gender_category_table = pd.crosstab(index = sales_clean.category, columns = sales_clean.sex)
gender_category_table

In [ ]:
gender_category_table.plot(kind="bar", rot= 0);

In [ ]:
stat, p, dof, expected = sci.chi2_contingency(gender_category_table)
if p > 0.01:
    print("Gender and Category are probably independent")
else:
    print("Gender and Category are probably dependent")

There are more sales in category 0 and very less in category 2, even if category sales has higher prices.   
In category **0** *Male* have more sales than *Female*.  
In category **1** *Female* have more sales than *Male*.  
In category **2** *Male* have more sales than *Female*. 

**Correlation between age and total amount of purchases**

In [ ]:
# Create a dataframe with age and the total amount purchases
corr_age_price = sales_clean.groupby("age").sum().reset_index()
# Plot using regplot
sns.regplot(x = "age", y = "price", data = corr_age_price, fit_reg = True);
sci.pearsonr(sales_clean.price, sales_clean.age)

There is a weak correlation between the age and the amount of purchases, as the age increases the amount of purchases decreases.  
Young people buy more than old people. 

**Correlation between age and the purchase frequency(Number of purchases per month)**

In [ ]:
sales_clean_freq = sales_clean.copy()
sales_clean_freq["day"] = sales_clean_freq.date.dt.month
g = sales_clean_freq.groupby(["day", "age"]).id_prod.count().reset_index()
sns.relplot("age", "id_prod", data = g, kind = "scatter", col = "day")

**Correlation between age and the categories of purchases products**

**Correlation between age and the average basket size (in number of items)**